In [2]:
import pandas as pd
import sqlite3

In [3]:
orders = pd.read_csv("data/orders.csv")
users = pd.read_json("data/users.json")

In [9]:
import os
os.getcwd()

'E:\\all projects\\food_delivery'

In [10]:
os.listdir()

['.ipynb_checkpoints',
 'analysis.ipynb',
 'data',
 'README.md',
 'requirements.txt']

In [11]:
os.listdir()

['.ipynb_checkpoints',
 'analysis.ipynb',
 'data',
 'README.md',
 'requirements.txt',
 'restaurants.sql']

In [12]:
conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [13]:
df = orders.merge(users, on="user_id") \
           .merge(restaurants, on="restaurant_id")

In [14]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [15]:
df.groupby("cuisine")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [16]:
user_spend = df.groupby("user_id")["total_amount"].sum()

count_users = user_spend[user_spend > 1000].count()
count_users

2544

In [17]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

df.groupby("rating_range")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)

rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [18]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [19]:
cuisine_summary = df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
)

cuisine_summary.sort_values(["distinct_restaurants", "revenue"],
                            ascending=[True, False])


,distinct_restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [20]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage

50

In [21]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_7964\2916288219.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [23]:
df[df["membership"] == "Gold"].shape[0]

4987

In [24]:
round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)

1889367

In [25]:
df["user_id"].nunique()

2883

In [26]:
round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

797.15

In [27]:
df[df["rating"] >= 4.5].shape[0]

3374